In [1]:
# training
# A neural network implemented in PyTorch.
# At least one other Scikit-learn algorithm (for example, K-NN, SVM, Random Forest, Logistic Regression, etc.).
import pandas as pd
import numpy as np
from sklearn import neighbors

In [16]:
traindata = pd.read_csv('Datasets/PractiseData/development.csv', header=0)

ytrain = traindata.iloc[:, :]
print(ytrain)


                                  label  gender  profession ideology_binary  \
0      00369358fac3b8d42845f82f0c3ececc    male  journalist            left   
1      00369358fac3b8d42845f82f0c3ececc    male  journalist            left   
2      00369358fac3b8d42845f82f0c3ececc    male  journalist            left   
3      00369358fac3b8d42845f82f0c3ececc    male  journalist            left   
4      00369358fac3b8d42845f82f0c3ececc    male  journalist            left   
...                                 ...     ...         ...             ...   
14395  ffcc655c22b6d8b96857db135b61b61a  female  journalist           right   
14396  ffcc655c22b6d8b96857db135b61b61a  female  journalist           right   
14397  ffcc655c22b6d8b96857db135b61b61a  female  journalist           right   
14398  ffcc655c22b6d8b96857db135b61b61a  female  journalist           right   
14399  ffcc655c22b6d8b96857db135b61b61a  female  journalist           right   

      ideology_multiclass                          

## Exploratory Analysis of the Dataset

Before applying language processing techniques, a exploratory analysis of the dataset is performed. This analysis includes:
- General description: Number of instances, variables, data types, missing values
- Basic statistics and simple visualizations (class distribution, text lengths, etc.)
- Preliminary text analysis (most frequent words, word cloud, examples by class)
- Formulation of initial hypoteses about possible relationships between disinfomation and polarization.

In [29]:
# Shape of the training data
print("Training data shape:", ytrain.shape)

# Print the headers of the dataset
print("Headers:", ytrain.columns.tolist())

# Print the data types of each column
for col in ytrain.columns:
    first_non_null = ytrain[col].dropna().iloc[0] if not ytrain[col].dropna().empty else None
    print(f"  Column: {col} -> type: {type(first_non_null).__name__}")

# Users are in the first column, extract unique users from ytrain
unique_users = ytrain.iloc[:, 0].unique()
print("Unique users (labels) in training data:", len(unique_users))


Training data shape: (14400, 6)
Headers: ['label', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass', 'tweet']
  Column: label -> type: str
  Column: gender -> type: str
  Column: profession -> type: str
  Column: ideology_binary -> type: str
  Column: ideology_multiclass -> type: str
  Column: tweet -> type: str
Unique users (labels) in training data: 360
